In [15]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('../data/source.csv')
df.head(10)

,Unnamed: 0,title,num_comments,post_upvotes,id,post_upvote_ratios,datetime,score,text,upvote_ratio,post_author,date,time
0,0,Frequently Asked Questions + Weekly Discussion...,11,23,101617w,0.94,2023-01-02 07:00:11,26,**Welcome to the Weekly Discussion. Please rea...,0.94,AutoModerator,2023-01-02,07:00:11
1,1,"Private data of 235,000,000 Twitter users leak...",55,348,103gwl6,0.94,2023-01-04 22:17:41,348,"The database contains 235,000,000 unique recor...",0.94,Malwarebeasts,2023-01-04,22:17:41
2,2,Seedshift - steganographically encrypt your se...,0,13,103v5od,0.93,2023-01-05 09:53:30,13,Pertaining to recent events (both the FTX coll...,0.93,toshiromiballza,2023-01-05,09:53:30
3,3,Are NFT Scammers Just As Creative as the Artis...,16,91,103bey7,0.92,2023-01-04 18:46:44,85,NaN,0.92,Sufficient-Bid-5716,2023-01-04,18:46:44
4,4,Accidentally transferred Tether to my coinbase...,3,3,1040ny2,1.00,2023-01-05 14:46:17,3,I did not notice that coinbase states “only se...,1.00,YUNGjuulstin,2023-01-05,14:46:17
5,5,"🍄 Stereum - Ethereum Node - ""2.0.0-rc.10"" Rele...",0,1,10411tm,1.00,2023-01-05 15:02:02,1,NaN,1.00,daGscheid,2023-01-05,15:02:02
6,6,Is there some obscure use case scenario where ...,9,7,103l3dj,0.70,2023-01-05 01:07:00,8,"I know.. I know...\n\n""it is impossible to syn...",0.70,cathie_burry,2023-01-05,01:07:00
7,7,A list of Crypto Friendly Banks,25,94,102yufr,0.82,2023-01-04 09:07:54,92,NaN,0.82,legragrug,2023-01-04,09:07:54
8,8,Eth scammer,2,1,103qmsf,0.56,2023-01-05 05:26:29,4,I’ve found a wallet involved in high stake sca...,0.56,PhilosopherEast6227,2023-01-05,05:26:29
9,9,Developing crypto,5,13,103agx5,0.85,2023-01-04 18:10:25,13,So I hear a lot of talk about: What have you g...,0.85,BlazingJava,2023-01-04,18:10:25
